In [21]:
from src.Model_Parameter import get_model_response

import re
import os
import csv
import json
from typing import List
import yaml
import openai
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import requests
import xml.etree.ElementTree as ET
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
!pip install fastparquet

     |████████████████████████████████| 911 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 11.8 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 341 kB 20.1 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


# Llama 2 - 7B

In [5]:
import pandas as pd
data = pd.read_parquet('llama2_7b_chat_hf_pubmed_summarization.parquet')

In [36]:
articles = pd.read_parquet('articles.parquet')

In [39]:
articles.head()

,article,abstract,record_id
0,gastric cancer is the second most frequent neo...,the typical symptoms of advanced cancer of the...,53
1,pathological gambling is defined in the curren...,abstractthe incidence of pathological gambling...,204
2,beyond its role of cellular energy currency an...,release and reception of extracellular atp by ...,217
3,generalized anxiety disorder ( gad ) is a chro...,objective : to elucidate the consequences of b...,244
4,sterilization in orthodontics has repeatedly b...,background : sterilization techniques could af...,260


In [57]:
data

,record_id,prompt,ground_truth,mapped_ground_true,model_response,answer,mapped_answer,time
0,5238,"\nAs a medical expert, you are given a set of ...","during the last years , self - assembled organ...","during the last years , self - assembled organ...",Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-11-16-48
1,912,"\nAs a medical expert, you are given a set of ...",aim . \n acute pancreatitis is an inflammatory...,aim . \n acute pancreatitis is an inflammatory...,"Great, thank you! Based on these four summarie...","Great, thank you! Based on these four summarie...","Great, thank you! Based on these four summarie...",11-27-2023-11-17-18
2,204,"\nAs a medical expert, you are given a set of ...",abstractthe incidence of pathological gambling...,abstractthe incidence of pathological gambling...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-11-18-18
3,6074,"\nAs a medical expert, you are given a set of ...",purposepulmonary hypertension and exercise - i...,purposepulmonary hypertension and exercise - i...,Here&#x27;s a consolidated summary of the four...,Here&#x27;s a consolidated summary of the four...,Here&#x27;s a consolidated summary of the four...,11-27-2023-11-19-22
4,2253,"\nAs a medical expert, you are given a set of ...",an 810-nm diode laser was used to non - surgic...,an 810-nm diode laser was used to non - surgic...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-11-19-55
...,...,...,...,...,...,...,...,...
95,584,"\nAs a medical expert, you are given a set of ...",oral cavity is an uncommon site for metastasis...,oral cavity is an uncommon site for metastasis...,"As a medical expert, I can consolidate the inf...","As a medical expert, I can consolidate the inf...","As a medical expert, I can consolidate the inf...",11-27-2023-12-43-25
96,4990,"\nAs a medical expert, you are given a set of ...",acute appendicitis in children with acute leuk...,acute appendicitis in children with acute leuk...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-12-44-15
97,5201,"\nAs a medical expert, you are given a set of ...",eye injury is a significant disabling worldwid...,eye injury is a significant disabling worldwid...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-12-45-40
98,1401,"\nAs a medical expert, you are given a set of ...",lead \n is a toxic heavy metal whose detoxific...,lead \n is a toxic heavy metal whose detoxific...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-12-47-17


In [41]:
merged_df = pd.merge(data, articles, on='record_id')

In [56]:
merged_df

,record_id,prompt,ground_truth,mapped_ground_true,model_response,answer,mapped_answer,time,article,abstract
0,5238,"\nAs a medical expert, you are given a set of ...","during the last years , self - assembled organ...","during the last years , self - assembled organ...",Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-11-16-48,"during the last years , the self - assembly of...","during the last years , self - assembled organ..."
1,912,"\nAs a medical expert, you are given a set of ...",aim . \n acute pancreatitis is an inflammatory...,aim . \n acute pancreatitis is an inflammatory...,"Great, thank you! Based on these four summarie...","Great, thank you! Based on these four summarie...","Great, thank you! Based on these four summarie...",11-27-2023-11-17-18,acute pancreatitis is an inflammatory intra - ...,aim . \n acute pancreatitis is an inflammatory...
2,204,"\nAs a medical expert, you are given a set of ...",abstractthe incidence of pathological gambling...,abstractthe incidence of pathological gambling...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-11-18-18,pathological gambling is defined in the curren...,abstractthe incidence of pathological gambling...
3,6074,"\nAs a medical expert, you are given a set of ...",purposepulmonary hypertension and exercise - i...,purposepulmonary hypertension and exercise - i...,Here&#x27;s a consolidated summary of the four...,Here&#x27;s a consolidated summary of the four...,Here&#x27;s a consolidated summary of the four...,11-27-2023-11-19-22,exacerbations of copd are associated with acce...,purposepulmonary hypertension and exercise - i...
4,2253,"\nAs a medical expert, you are given a set of ...",an 810-nm diode laser was used to non - surgic...,an 810-nm diode laser was used to non - surgic...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-11-19-55,peri - implantitis is inflammation of the peri...,an 810-nm diode laser was used to non - surgic...
...,...,...,...,...,...,...,...,...,...,...
95,584,"\nAs a medical expert, you are given a set of ...",oral cavity is an uncommon site for metastasis...,oral cavity is an uncommon site for metastasis...,"As a medical expert, I can consolidate the inf...","As a medical expert, I can consolidate the inf...","As a medical expert, I can consolidate the inf...",11-27-2023-12-43-25,"nearly , 90% of metastatic tumors occur in jaw...",oral cavity is an uncommon site for metastasis...
96,4990,"\nAs a medical expert, you are given a set of ...",acute appendicitis in children with acute leuk...,acute appendicitis in children with acute leuk...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-12-44-15,acute appendicitis and typhlitis are the most ...,acute appendicitis in children with acute leuk...
97,5201,"\nAs a medical expert, you are given a set of ...",eye injury is a significant disabling worldwid...,eye injury is a significant disabling worldwid...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-12-45-40,eye injury is a significant health problem wor...,eye injury is a significant disabling worldwid...
98,1401,"\nAs a medical expert, you are given a set of ...",lead \n is a toxic heavy metal whose detoxific...,lead \n is a toxic heavy metal whose detoxific...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,Sure! Here&#x27;s a consolidated summary of th...,11-27-2023-12-47-17,lead \n poisoning is one of the most serious e...,lead \n is a t

In [48]:
correct_summary = merged_df['mapped_ground_true'].tolist()
model_generated = merged_df['mapped_answer'].tolist()

In [53]:
articles = merged_df['article'].tolist()

In [49]:
print(len(correct_summary))
len(model_generated)

100


100

In [50]:
from rouge_score import rouge_scorer

In [ ]:
# Summarize osmosis transcripts and evaluate
import time

# Start time
start_time = time.time()

#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
for i, note in enumerate(correct_summary):
    
    # Evaluate BLEU score
    bleu_score = corpus_bleu([[correct_summary[i].split()]], [model_generated[i].split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = correct_summary[i]
    hypothesis = model_generated[i]
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    #print("Clinical Note:")
    #print(note)
    print("\nCorrect Summary:")
    print(correct_summary[i])
    print("\nGenerated Summary:")
    print(model_generated[i])
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

    
end_time = time.time()

In [52]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.019981344079153505
Average ROUGE-1 Score: 0.34797439474519765
Average ROUGE-2 Score: 0.10061489198211244
Average ROUGE-L Score: 0.18590620129783425
Average METEOR Score: 0.24134565111944753


In [55]:
print(len(articles))
print(len(correct_summary))
print(len(model_generated))
print(len(meteor_scores))

100
100
100
100


In [58]:
!pip install pandas openpyxl

In [59]:
data = {
    'articles': articles,
    'correct_summary': correct_summary,
    'model_generated': model_generated,
    'meteor_scores': meteor_scores
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Exporting the DataFrame to an Excel file
df.to_excel('llama2_output.xlsx', index=False)